In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import collections
import numpy as np
import re
from functions import logistic_reg,xg_boost,process_data_rs,run_xgboost,resultado,run_logreg
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
dates = ['DATA_SINTOMAS','DATA_CONFIRMACAO','DATA_EVOLUCAO','DATA_EVOLUCAO_ESTIMADA']
df = pd.read_csv('20200702.csv',sep=';',encoding='iso-8859-1',parse_dates=dates)

len(df.columns)

print('Casos Totais: {}'.format(df['HOSPITALIZACAO'].value_counts().sum()))
print('Casos Em Acompanhamento: {}'.format(df.loc[df['EVOLUCAO'] == 'EM ACOMPANHAMENTO']['EVOLUCAO'].value_counts().sum()))
print('Casos Hospitalizados:{}'.format(df['HOSPITALIZACAO'].value_counts().values[1]))


Casos Totais: 28171
Casos Em Acompanhamento: 4240
Casos Hospitalizados:3454


In [3]:
process_data = process_data_rs(df)
process_data = process_data.loc[process_data['EVOLUCAO'] != 'EM ACOMPANHAMENTO']
process_data = process_data.drop(columns=['EVOLUCAO'])



In [4]:
data = model = pickle.load(open("cols.pkl", 'rb'))
cols = list(process_data.columns.values)

for x in data:
    if not x in cols:
        process_data[x] = 0
        
for x in cols:
    if not x in data:
        process_data = process_data.drop(columns=[x])
        


In [5]:
len(process_data.columns)

479

In [6]:
  y = 'HOSPITALIZACAO'
  X = process_data.drop(columns=y).to_numpy()
  Y = process_data[y].to_numpy()
  
  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=1)

In [7]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D


model_dense = Sequential()
model_dense.add(Dense(64, input_dim=478, activation='relu'))
model_dense.add(Dense(32, activation='relu'))
model_dense.add(Dense(16, activation='relu'))
model_dense.add(Dense(4, activation='relu'))
model_dense.add(Dense(1, activation='sigmoid'))
model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


In [8]:
history = model_dense.fit(x_train, y_train, epochs=100, batch_size=64)


Epoch 1/100
20341/20341 [==============================] - 1s 73us/step - loss: 0.5796 - accuracy: 0.8879
Epoch 2/100
20341/20341 [==============================] - 1s 53us/step - loss: 0.1450 - accuracy: 0.9603
Epoch 3/100
20341/20341 [==============================] - 1s 54us/step - loss: 0.1357 - accuracy: 0.9624
Epoch 4/100
20341/20341 [==============================] - 1s 55us/step - loss: 0.1280 - accuracy: 0.9645
Epoch 5/100
20341/20341 [==============================] - 1s 53us/step - loss: 0.1216 - accuracy: 0.9648
Epoch 6/100
20341/20341 [==============================] - 1s 53us/step - loss: 0.1170 - accuracy: 0.9654
Epoch 7/100
20341/20341 [==============================] - 1s 54us/step - loss: 0.1120 - accuracy: 0.9662
Epoch 8/100
20341/20341 [==============================] - 1s 53us/step - loss: 0.1082 - accuracy: 0.9665
Epoch 9/100
20341/20341 [==============================] - 1s 53us/step - loss: 0.1045 - accuracy: 0.9674
Epoch 10/100
20341/20341 [====================

In [9]:
eval_model=model_dense.evaluate(x_train, y_train)
eval_model
model_dense.save('dense_ network')

20341/20341 [==============================] - 1s 41us/step


In [10]:
y_pred= model_dense.predict(x_test)
y_pred =(y_pred>0.5).astype(int)[:,0]

In [11]:
resultado(y_test,y_pred),

Numero de 0: 3187, %: 0.8877437325905293 
Numero de 1: 403, %: 0.11225626740947076

Accuracy: 96.32%
Resultado
Correto Negativo    3116
Correto Positivo     342
Falso Negativo        71
Falso Positivo        61
Name: predict, dtype: int64

Taxa de Acerto Negativos:97.8% 
Taxa de Acerto Positivos: 84.9%


(None,)

# **Conv net**

In [12]:
x_train = x_train.reshape(20341,478,1,1)
x_test = x_test.reshape(3590,478,1,1)

In [13]:


conv_net = Sequential()
conv_net.add(Conv2D(32, kernel_size=(1, 1),
                 activation='relu',
                 input_shape=(478,1,1)))
conv_net.add(Conv2D(64, (1, 1), activation='relu'))
conv_net.add(MaxPooling2D(pool_size=(1, 1)))
conv_net.add(Dropout(0.25))
conv_net.add(Flatten())
conv_net.add(Dense(128, activation='relu'))
conv_net.add(Dropout(0.5))
conv_net.add(Dense(1, activation='sigmoid'))
conv_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
history = conv_net.fit(x_train, y_train, epochs=100, batch_size=64)


Epoch 1/100
20341/20341 [==============================] - 3s 151us/step - loss: 0.5758 - accuracy: 0.9510
Epoch 2/100
20341/20341 [==============================] - 2s 87us/step - loss: 0.3753 - accuracy: 0.9635
Epoch 3/100
20341/20341 [==============================] - 2s 86us/step - loss: 0.1834 - accuracy: 0.9644
Epoch 4/100
20341/20341 [==============================] - 2s 87us/step - loss: 0.1225 - accuracy: 0.9665
Epoch 5/100
20341/20341 [==============================] - 2s 87us/step - loss: 0.1207 - accuracy: 0.9674
Epoch 6/100
20341/20341 [==============================] - 2s 86us/step - loss: 0.1024 - accuracy: 0.9661
Epoch 7/100
20341/20341 [==============================] - 2s 86us/step - loss: 0.0897 - accuracy: 0.9668
Epoch 8/100
20341/20341 [==============================] - 2s 87us/step - loss: 0.0874 - accuracy: 0.9661
Epoch 9/100
20341/20341 [==============================] - 2s 85us/step - loss: 0.0852 - accuracy: 0.9670
Epoch 10/100
20341/20341 [===================

In [15]:
eval_model=conv_net.evaluate(x_train, y_train)
eval_model
conv_net.save('conv_net')

20341/20341 [==============================] - 1s 54us/step


In [16]:
y_pred= conv_net.predict(x_test)
y_pred =(y_pred>0.5).astype(int)[:,0]

In [17]:
resultado(y_test,y_pred),

Numero de 0: 3199, %: 0.8910863509749304 
Numero de 1: 391, %: 0.10891364902506964

Accuracy: 96.82%
Resultado
Correto Negativo    3131
Correto Positivo     345
Falso Negativo        68
Falso Positivo        46
Name: predict, dtype: int64

Taxa de Acerto Negativos:97.9% 
Taxa de Acerto Positivos: 88.2%


(None,)